# Keybert + TF-IDF

In [2]:
import pandas as pd
import numpy as np

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

# keybert
from keybert import KeyBERT

In [31]:
date_str = '20250814'

In [32]:
# 파일 불러오기
raw_data_gen = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_general_{date_str}.csv")
raw_data_i = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_i_{date_str}.csv")
raw_data_g = pd.read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/hankyung_g_{date_str}.csv")

# 하나의 파일로 유니온
raw_data = pd.concat([raw_data_gen, raw_data_i, raw_data_g], ignore_index=True)
print(raw_data.shape)

# 완전일치 중복기사 제거
df = raw_data.drop_duplicates(subset=['title', 'text'], keep='first').copy()

# title에 [속보] 또는 [포토] 포함 시 text를 NaN으로 변경
df.loc[df['title'].str.contains(r'\[속보\]|\[포토\]', regex=True), 'text'] = np.nan

# null셀을 빈칸으로 만들기
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# 특수문자 등 전처리
df['text_clean'] = df['text'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df['title_clean'] = df['title'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df.head(3)

(343, 4)


,url,title,text,publish_date,text_clean,title_clean
0,https://www.hankyung.com/article/2025081410044,한국경제,미국의 7월 소비자물가(CPI)가 발표된 뒤 어제 S&P500 지수는 올해 16번째...,NaN,미국의 월 소비자물가 CPI 가 발표된 뒤 어제 S&P 지수는 올해 번째...,한국경제
5,https://www.hankyung.com/article/2025081410247,"'1000만 관객 동원' 40대 배우, 자택서 아내 때려 신고당해",사진은 기사 내용과 관련 없음./사진=게티이미지뱅크\n\n40대 배우가 자택에서 아...,2025-08-14 14:21:27+09:00,사진은 기사 내용과 관련 없음 사진 게티이미지뱅크 대 배우가 자택에서 아내를...,만 관객 동원 대 배우 자택서 아내 때려 신고당해
6,https://www.hankyung.com/article/2025081410266,"이찬진 신임 금감원장 ""주가조작 엄정 대응…상법개정안 안착 지원""",이찬진 신임 금감원장 취임식\n\n이찬진(오른쪽) 신임 금융감독원장 /사진=금감원 ...,2025-08-14 15:00:01+09:00,이찬진 신임 금감원장 취임식 이찬진 오른쪽 신임 금융감독원장 사진 금감원 제공...,이찬진 신임 금감원장 주가조작 엄정 대응 상법개정안 안착 지원


In [10]:
corpus = """
"삼성도 제쳤다"<br>은행원 연봉, '상반기에만 6350만원'
"연 1000억 더 낸다"
정부 발표에 카드업계 난리 난 까닭
"연 1000억 더 낸다"<br>정부 발표에 카드업계 난리 난 까닭
메리츠화재, 손보 1위 등극…교보는 '2위 생보사' 굳히기
메리츠화재, 손보 1위 등극…교보는 '2위 생보사' 굳히기
투톱 앉혔다…다시 돌아가는 금융권 '인사 시계'
금융위원장, 금융감독원장 등 ‘투톱’ 자리가 채워져 멈춰 있던 금융권 ‘인사 시계’가 다시 돌아갈 것으로 예상된다. 현재 공석인 금융당국 임원급 보직을 시작으로 국책은행장, 금융공공기관장 및 협회장에 이르기까지 대대적인 ‘인사 태풍’이 몰아칠 것이라는 관측이 나온다.…

투톱 앉혔다…다시 돌아가는 금융권 '인사 시계'
국민·신한銀 "비대면 IRP 가입, 수수료 無"
국민은행과 신한은행이 나란히 개인형퇴직연금(IRP) 가입자를 상대로 수수료 면제 전략을 내걸었다.국민은행은 오는 10월 비대면으로 IRP에 가입해 5000만원 이상 넣는 고객에게 모든 수수료(연 0.38%)를 면제한다고 14일 발표했다. 적립금이 5000만원 미만인 …

금융정책

HMM 2조 자사주 매입·소각 지분 판 산은, 公자금 첫 회수
HMM이 약 2조2000억원을 들여 자사주 매입과 소각에 나선다. 전체 발행 주식의 8%에 달하는 대규모 매입·소각이다. 산업은행은 보유 지분을 HMM에 팔아 기존에 투입한 공적자...

HMM 2조 자사주 매입·소각 지분 판 산은, 公자금 첫 회수
이억원 "생산적 금융으로 대전환…금감원과는 원팀 관계"
이재명 정부 첫 금융위원장에 지명된 이억원 후보자(사진)는 “한국 경제가 진짜 성장과 재도약을 하기 위해 그 어느 때보다 금융의 역할이 매우 중요한 상황”이라며 “생산적 금융 대전...

이억원 "생산적 금융으로 대전환…금감원과는 원팀 관계"
이찬진 "상생지수 도입해 모험자본 활성화"
이억원 금융위원장 후보자 "이찬진 금감원장에 '원팀 협업' 제안"
이찬진 금감원장 "나는 호가호위와 거리 먼 사람…업계 의견 경청할 것"
'거시경제 정책통' 이억원 후보자 …이재명표 금융정책 '속도'
민변 출신…李와 연수원 동기 "금융권 의견 열심히 경청할 것"
금융위원장에 이억원 전 기재차관…금감원장에 이찬진 변호사 내정
금융위원장에 이억원, 금감원장에 이찬진 변호사 지명
금융위, 오늘 임시 금융위 개최…신임 금감원장 임명 제청될 듯
은행
KB금융, 명예를 품은 가게 현판식
KB금융그룹은 지난 13일 서울 창동에서 첫 번째 ‘명예를 품은 가게’(명품가게) 현판식을 열었다. 명품가게는 KB금융이 광복 80주년을 맞아 독립유공자 후손 소상공인의 자립을 지...

KB금융, 명예를 품은 가게 현판식
토스뱅크 목돈굴리기 출시 3주년…누적 연계액 18조 육박
인터넷은행 토스뱅크는 ‘목돈굴리기’의 누적 연계 금액이 18조 원에 육박한다고 14일 밝혔다. 토스뱅크가 2022년 8월 목돈굴리기 서비스를 처음 출시한 이후 3년 만의 성과다.토...

토스뱅크 목돈굴리기 출시 3주년…누적 연계액 18조 육박
적금인데 이자 매월지급…케이뱅크, '데굴데굴 농장' 출시
인터넷은행 케이뱅크는 적금의 이자를 만기까지 기다릴 필요 없이 매달 이자를 받는 새로운 형태의 적금 상품 ‘데굴데굴 농장’을 출시했다고 14일 발표했다.‘데굴데굴 농장’은 매월 최...

적금인데 이자 매월지급…케이뱅크, '데굴데굴 농장' 출시
신한금융그룹, 건국대와 청소년 진로 상담
신한금융그룹은 지난 12일 건국대에서 전국 지역아동센터 청소년을 상대로 진로 상담을 했다. 진옥동 회장(오른쪽)을 비롯한 신한금융 임직원, 원종필 총장(왼쪽)을 포함한 건국대 임직...

신한금융그룹, 건국대와 청소년 진로 상담
하나은행, BIDV와 QR결제 동맹
하나은행은 지난 12일 서울 롯데호텔 본관에서 열린 ‘한국·베트남 비즈니스 포럼’에서 베트남투자개발은행(BIDV)과 국가 간 QR결제 서비스 활성화를 위한 업무협약을 맺었다고 13...

하나은행, BIDV와 QR결제 동맹
케이뱅크, 2분기 최대 순익 달성…이자이익 감소에도 '깜짝 실적'
인터넷은행 케이뱅크가 지난 2분기 700억원에 육박하는 순이익을 내며 분기 단위 최대 실적 기록을 썼다. 이자이익이 20% 가까이 줄었지만 비이자이익이 늘고 대손비용이 감소하면서 ...

케이뱅크, 2분기 최대 순익 달성…이자이익 감소에도 '깜짝 실적'
보험·2금융
삼성생명, 서울 4개 대학과 산학협력
삼성생명은 지난 8일 서울 서초동 삼성금융캠퍼스에서 라이프놀로지랩 2기 첫 워크숍(사진)을 열었다고 14일 발표했다. 라이프놀로지랩은 인생(life)과 기술(technology)을...

삼성생명, 서울 4개 대학과 산학협력
ABL생명, '여름나기 꾸러미' 제작 봉사
ABL생명은 지난 13일 서울 수색동 은평종합사회복지관에서 취약계층을 위한 ‘여름나기 꾸러미’를 만드는 봉사활동(사진)을 하고 기부금 1000만원을 초록우산에 전달했다. 기부금은 ...

ABL생명, '여름나기 꾸러미' 제작 봉사
"이게 되네"…버스 타고 제주도 여행하려던 외국인 '화들짝'
제주도를 방문하는 외국인 관광객은 이제 별도의 교통가드를 발급받을 필요 없이 자국에서 사용하던 비자 카드만 있으면 간편하게 버스를 이용할 수 있게 됐다. 비자(Visa)가 지난 1...

"이게 되네"…버스 타고 제주도 여행하려던 외국인 '화들짝'
뱅크샐러드, 2분기 첫 흑자 전환…매출 85% 급성장
마이데이터 기반 자산관리 서비스를 제공하는 뱅크샐러드가 지난 2분기 처음으로 흑자를 냈다. 뱅크샐러드는 올 하반기 마이데이터 2.0 인프라를 활용해 새로운 금융·건강 관리 서비스를...

뱅크샐러드, 2분기 첫 흑자 전환…매출 85% 급성장
건전성 '경고등'…보험사 자본규제 확 푼다
금융당국이 당초 강화하기로 계획한 ‘보험부채 할인율 규제’를 확 푸는 쪽으로 가닥을 잡았다. 올 들어 보험사 지급여력(K-ICS·킥스) 비율이 급락하자 업계에서 규제 완화를 요구하...

건전성 '경고등'…보험사 자본규제 확 푼다
KB증권, 디폴트옵션·TDF로 연금 수익률 높인다
올해로 도입 20주년을 맞은 퇴직연금 시장이 놀라운 성장세를 이어가고 있다. 지난 10년 간 개인이 직접 운용하는 확정기여형(DC형) 퇴직연금과 개인형 퇴직연금의 적립금은 각각 연...

KB증권, 디폴트옵션·TDF로 연금 수익률 높인다
가상자산·핀테크
한 달 만에 역대 최고가 찍더니…심상치 않은 '비트코인 랠리'
비트코인이 사상 최고치를 경신했다. 미 중앙은행(Fed)의 금리 인하 기대가 퍼지면서 투자심리가 살아난 영향으로 분석된다.16일 국내 암호화폐거래소 업비트에 따르면 비트코인은 지난...

한 달 만에 역대 최고가 찍더니…심상치 않은 '비트코인 랠리'
'진격의 이더리움' 어디까지 오를까…사상 최고가 눈앞
업비트, 베트남 밀리터리뱅크와 제휴
"모든 것은 암호화 될 것"…크립토 산업에서 새 일자리 나온다
업비트, 베트남 밀리터리뱅크와 제휴…가상자산거래소 설립 협력
재테크
"여보, 그거 챙겼지?"…요즘 난리 난 해외여행 '필수템'
여름 휴가철을 맞아 무료 환전 혜택을 제공하는 트래블카드에 대한 관심이 커지고 있다. 은행권과 카드사들은 지난해 환전 등에 붙는 각종 수수료를 일제히 없앤 데 이어 올해는 결제 과...

"여보, 그거 챙겼지?"…요즘 난리 난 해외여행 '필수템'
하루 만원씩 '따박따박' 한달 부었더니…'역대급 이자'
미래에셋증권, 맞춤형 포트폴리오…출시 2년 운용액 3조 넘어
NH아문디운용, 국내 원전기업 분산투자…순자산 3000억 돌파
연 16.5% 수익 보장 '연금저축'…50대에 가입해도 늦지 않아

"공짜라서 일단 신청했는데…"
정부, '국비교육' 손본다
"공짜라서 일단 신청했는데…" <br>정부, '국비교육' 손본다
"월 119만원 감당 안돼요"…아파트 전세 씨마르자 '한숨'
"월 119만원 감당 안돼요"…아파트 전세 씨마르자 '한숨'
정부 "10월 10일 임시공휴일 지정 없다…해외여행 수요만 커져"
정부 "10월 10일 임시공휴일 지정 없다…해외여행 수요만 커져"
첫아이 출산장려금 효과 미미…"사교육·주거비 구조개혁 절실"
첫아이 출산장려금 효과 미미…"사교육·주거비 구조개혁 절실"


경제정책


'10일 연휴' 물건너 가나…기재부 "임시공휴일 지정, 상상도 안 해"
이재명 대통령이 지난 14일 “10월 긴 추석 연휴 등을 활용한 내수 활성화 방안을 선제적으로 강구해 달라”고 주문함에 따라 오는 10월 10일을 임시 공휴일로 지정하고, 3차 추...

'10일 연휴' 물건너 가나…기재부 "임시공휴일 지정, 상상도 안 해"
보조금 퍼주더니 '날벼락'…"돈 내고 팝니다" 초유의 상황
1950년대 중반 전 세계는 핵에너지에 대한 낙관론으로 들썩였다. 루이스 스트로스 당시 미국 원자력위원회 위원장은 "원전으로 인해 전기를 계량할 필요도 없을 만큼 전기값이 싸질 수...

보조금 퍼주더니 '날벼락'…"돈 내고 팝니다" 초유의 상황
"석화 구조개편 무임승차 기업 엄중 대응"
정부가 공급 과잉으로 공멸 위기에 처한 석유화학업계 구조조정에 본격적으로 칼을 빼들었다. 이달 말 ‘석유화학 구조개편 종합대책’ 발표를 앞두고 기업들에 ‘뼈를 깎는 자구 노력’을 ...

"석화 구조개편 무임승차 기업 엄중 대응"
'물과 기름' 과기부·산업부, 제조AI 육성에 손잡는다
첨단기술 분야에서 관할권을 두고 크고 작은 갈등을 겪은 과학기술정보통신부와 산업통상자원부가 인공지능전환(AX)을 위해 손을 잡기로 했다. 첫 번째 과제로 지역 AX 선도모델을 구축...

'물과 기름' 과기부·산업부, 제조AI 육성에 손잡는다
'유류세 인하' 10월 말까지 연장
'예비타당성조사' 기준 500억→1000억원 완화
주병기 "경제적 강자 甲질 막겠다"…온플법은 속도조절
일반수소발전 경쟁입찰 결과 "낙찰물량 88% 수도권·광역시 집중"


거시경제


올 2분기 美·日·中 수출액 일제히 감소…EU·동남아에서 늘어
지난 2분기 한국의 핵심 교역 상대인 미국·일본·중국에 대한 수출액이 일제히 줄어든 것으로 나타났다. 대기업과 중견기업의 수출이 부진한 결과로 분석된다. 기업 규모별로는 중견·중소...

올 2분기 美·日·中 수출액 일제히 감소…EU·동남아에서 늘어
지방 균형발전에 60조 투자…국회 본원·대통령 집무실 세종 이전
정부가 지역 균형 발전을 위해 국세와 지방세 수입 비율을 7 대 3으로 개선한다. 사회적 합의를 거쳐 국회 본원과 대통령 집무실도 세종시로 옮긴다.대통령 직속 국정기획위원회는 13...

"상장사 보고서 절반이 오류"…'XBRL' 무용지물 전락하나
▶마켓인사이트 8월 13일 오전 9시 50분상장사가 공시한 재무보고서 절반이 국제표준에 맞지 않아 컴퓨터가 데이터를 제대로 읽어내지 못하는 것으로 나타났다. 국제표준 전산언어(XB...

올해 30% 뛴 銀…40弗 돌파 눈앞
국제 은값이 올해 들어 30% 넘게 오르며 상승세를 지속하고 있다. 공급 부족과 산업재 수요 증가가 겹친 영향이다. 금값이 급등해 은과 격차가 벌어진 것도 상승 요인이다. 글로벌 ...

올해 30% 뛴 銀…40弗 돌파 눈앞
"그냥 쉬었어요" 20대만 42만명…7월 기준 역대 최대
2차 추경 집행에도…KDI "올해 0.8% 성장 그칠 것"
대한상의, 韓-베 비즈니스 포럼
"일관된 정책·기업 참여…日 밸류업 성공비결"
한경 외환시장 워치
관세에 급등했던 환율…미 고용지표에 급락
관세에 급등했던 환율…미 고용지표에 급락
강진규의 BOK워치
한은 "구조개혁 없이 통화정책 힘들다"…'오지랖' 논란 반박
한은 "구조개혁 없이 통화정책 힘들다"…'오지랖' 논란 반박
관가 포커스
"지속가능성 걱정된다"…분위기 확 바뀐 정부의 예산지침
"지속가능성 걱정된다"…분위기 확 바뀐 정부의 예산지침
외환시장
외평채 발행 효과…외환보유액 11억달러 증가
지난달 외환보유액이 11억달러 넘게 증가했다. 외화 외국환평형기금채권 발행에 성공하고, 외화자산 운용수익이 늘어난 영향이다.5일 한국은행에 따르면 지난달 말 기준 한국의 외환보유액...

외평채 발행 효과…외환보유액 11억달러 증가
환율 1400원 돌파…두달여 만에 '최고'
외국인이 한국 증시에서 대거 빠져나가면서 원·달러 환율이 1400원 위로 급등(원화 가치 급락)했다. 미국의 인플레이션 우려가 확산하면서 나타난 강달러 현상도 환율 상승에 영향을 ...

환율 출렁이자 외환거래 급증…일평균 821억달러 '역대 최대'
지난 2분기 외환 거래액이 역대 최대치를 경신한 것으로 나타났다. 원·달러 환율 변동성이 커지면서 거래가 활발하게 이뤄졌다. 외국인의 국내 주식투자가 늘어난 것도 영향을 줬다.29...

환율 출렁이자 외환거래 급증…일평균 821억달러 '역대 최대'
코스피에 외국인 훈풍…원·달러 환율 1380원대로 하락
외국인 투자자의 국내 주식 매수세가 강하게 나타나면서 원·달러 환율이 하락(원화 가치는 상승)했다. 3거래일 간 이어진 환율 상승세가 하락으로 전환하면서 환율이 1380원대로 내려...

코스피에 외국인 훈풍…원·달러 환율 1380원대로 하락
양호한 미국 경제…달러당 1400원 턱 밑까지 오른 환율
비트코인 사상 최고 경신하자 리플도 3달러 돌파
외환거래량 1위 하나은행, 야간거래 비중은 농협이 가장 높아
달러 약세에 불어난 외환보유액…4100억달러 복귀
세금
소액주주라고 안심했는데…"하마터면 세금 더 낼 뻔"
소액주주라고 안심했는데…"하마터면 세금 더 낼 뻔"
대한성형외과의사회 "외국인 미용의료 부가세 환급 특례 연장해야"
대한성형외과의사회 "외국인 미용의료 부가세 환급 특례 연장해야"
"매달 60만원 더 썼는데…" 딸 키우는 직장인 환호한 까닭
"매달 60만원 더 썼는데…" 딸 키우는 직장인 환호한 까닭
"20만원 내고 이게 맞아?"…서울 직장인 '깜짝' 놀란 사연
"20만원 내고 이게 맞아?"…서울 직장인 '깜짝' 놀란 사연
고용복지
"알바 부탁 들어줬다가 전과자 됐어요"…식당 사장의 눈물
"알바 부탁 들어줬다가 전과자 됐어요"…식당 사장의 눈물
도로공사 하청업체 30대 직원, 벌목하다 나무에 깔려 숨져
도로공사 하청업체 30대 직원, 벌목하다 나무에 깔려 숨져
"산재사망 반복땐 인허가 말소" 초강수 둔 정부
"산재사망 반복땐 인허가 말소" 초강수 둔 정부
"산재 재발 건설사 등록 말소"…초강력 대책 내놓은
"""

In [12]:

# 1. 정규식 클리닝 함수
def regex_clean(text):
    if not isinstance(text, str):
        return ''
    return re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', text)

# 2. 불용어 불러오기
with open("/Users/leesangwon/Documents/ThemeStock_file/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read().splitlines()

# 3. 불용어 제거 함수
def remove_stopwords(text_clean):
    if not isinstance(text_clean, str):
        return ''
    return ' '.join([word for word in text_clean.split() if word not in stopwords])

# 4. 조사 제거 함수
def remove_josa(text):
    if not isinstance(text, str):
        return ''
    return ' '.join([word for word, pos in okt.pos(text) if pos != 'Josa'])

# 5. DataFrame 변환
df = pd.DataFrame({'text': corpus.split("\n")})

# 6. 정규식 → 조사 제거 → 불용어 제거
df['text_clean'] = (
    df['text']
    .apply(regex_clean)       # 특수문자/숫자 제거
    .apply(remove_josa)       # 조사 제거
    .apply(remove_stopwords)  # 불용어 제거
)

print(df['text_clean'])

0                                     
1             삼성 제 쳤다 br 은행원 연봉 상반기 만원
2                             연 억 더 낸다
3                  정부 발표 카드 업계 난리 난 까닭
4      연 억 더 낸다 br 정부 발표 카드 업계 난리 난 까닭
                    ...               
200     도로공사 하청 업체 대 직원 벌목 하다 나무 깔려 숨져
201       산재 사망 반복 땐 인 허가 말소 초 강수 둔 정부
202       산재 사망 반복 땐 인 허가 말소 초 강수 둔 정부
203       산재 재발 건설 사 등록 말소 초 강력 대책 내놓은
204                                   
Name: text_clean, Length: 205, dtype: object


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1) 코퍼스 준비 (NaN 방지)
corpus = df['text_clean'].fillna('')

# 2) 벡터라이저 설정
# - tokenizer=lambda s: s.split(): 이미 공백으로 토큰화된 텍스트를 그대로 사용
# - lowercase=False: 한글/대소문 유지
# - ngram_range=(1,1): 단어 단위(필요 시 (1,2) 등으로 확장)
# - min_df, max_df, max_features는 데이터 규모에 맞게 조절
vectorizer = TfidfVectorizer(
    tokenizer=lambda s: s.split(),
    token_pattern=None,
    lowercase=False,
    ngram_range=(1,1),
    min_df=2,           # 2개 미만 문서에만 나온 단어 제거
    max_df=0.7,         # 70% 넘는 문서에 등장하는 흔한 단어 제거
    max_features=8000   # 상위 8천개 단어만 사용 (메모리 절약용)
)

# 3) 학습 및 변환 (희소행렬)
X = vectorizer.fit_transform(corpus)   # shape: (문서수, 단어수)
terms = vectorizer.get_feature_names_out()

# 4) (옵션) DataFrame으로 시각화 — 메모리 주의
# tfidf_df = pd.DataFrame.sparse.from_spmatrix(X, columns=terms)

# 5) 각 문서별 TF-IDF 상위 키워드 뽑기 함수
def top_tfidf_terms(row_index: int, topk: int = 10):
    row = X.getrow(row_index).toarray().ravel()
    if not np.any(row):
        return []
    idx = np.argsort(row)[::-1][:topk]
    return list(zip(terms[idx], row[idx]))

# 6) 모든 문서에 대해 상위 키워드 뽑아 df에 붙이기 (리스트→문자열)
topk = 10
df['tfidf_top_terms'] = [
    ', '.join([f'{t}:{w:.3f}' for t, w in top_tfidf_terms(i, topk)]) 
    for i in range(X.shape[0])
]

In [14]:
df.head()

,text,text_clean,tfidf_top_terms
0,,,
1,"""삼성도 제쳤다""<br>은행원 연봉, '상반기에만 6350만원'",삼성 제 쳤다 br 은행원 연봉 상반기 만원,"br:0.616, 삼성:0.616, 만원:0.490, 힘들다:0.000, 명예:0...."
2,"""연 1000억 더 낸다""",연 억 더 낸다,"낸다:0.596, 더:0.484, 연:0.459, 억:0.448, 힘들다:0.000..."
3,정부 발표에 카드업계 난리 난 까닭,정부 발표 카드 업계 난리 난 까닭,"난:0.395, 난리:0.395, 까닭:0.395, 카드:0.395, 업계:0.38..."
4,"""연 1000억 더 낸다""<br>정부 발표에 카드업계 난리 난 까닭",연 억 더 낸다 br 정부 발표 카드 업계 난리 난 까닭,"낸다:0.335, br:0.316, 난리:0.302, 까닭:0.302, 난:0.30..."


In [15]:
from collections import Counter
import re

# 1. 전체 키워드 모으기 (점수 제거)
all_terms = []
for row in df['tfidf_top_terms']:
    if not isinstance(row, str):
        continue
    # "단어:점수" → 단어만 추출
    terms = [re.split(r':', t)[0] for t in row.split(', ')]
    all_terms.extend(terms)

# 2. 빈도수 계산
term_freq = Counter(all_terms)

# 3. DataFrame으로 변환
term_freq_df = pd.DataFrame(term_freq.items(), columns=['term', 'count'])
term_freq_df = term_freq_df.sort_values(by='count', ascending=False).reset_index(drop=True)

term_freq_df.head(10)

,term,count
0,,45
1,마르,40
2,말,40
3,만원,39
4,리기,37
5,힘들다,35
6,말소,34
7,맞아,23
8,매달,22
9,를,21


In [17]:
term_freq_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/test_.csv", index=False)